<h2>Nested Cross Validation<h2>
<h3>Normal Cross-validation Problem<h3>
Overoptimistic Performance Estimate

1. Dataset used to select bast hyperparameter.
2. Lead to over optimistic estimation.
3. because, model is evaluated in data it has already seen during hyperparamater turning.


In [1]:
import pandas as pd

# Load the dataset
#data = pd.read_csv('HeightWeight.csv')

data = pd.read_csv('sin_noisy_80sample.csv')

# Extract features (X) and target variable (y)
#X = data['Height'].values.reshape(-1, 1)  # Reshape to 2D array
#y = data['Weight'].values

X = data[['x', 'x^2', 'x^3', 'x^4', 'x^5', 'x^6', 'x^7', 'x^8']]
X = X.to_numpy()
y = data['noisy_y'].values



FileNotFoundError: [Errno 2] No such file or directory: 'sin_noisy_80sample.csv'

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Resubstitution
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
resubstitution_rmse = np.sqrt(mean_squared_error(y, y_pred))

from sklearn.model_selection import cross_val_score

# Cross-validation
fold = 10
model = LinearRegression()
cv_scores = cross_val_score(model, X, y, cv=fold, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores).mean()

#output
print(f"Resubstitution RMSE: {resubstitution_rmse}")
print(f"CV RMSE: {cv_rmse:.2f}")


Resubstitution RMSE: 0.257766277865693
CV RMSE: 0.40


<h2>Compare Normal CV and Nested CV<h2>

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import mean_squared_error

model = LinearRegression()

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_scores = []

# Outer loop: split data into train and validation sets
for train_idx, val_idx in outer_cv.split(X):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Define the inner loop (for model evaluation)
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
    inner_rmse_scores = []

    # Inner loop: perform k-fold cross-validation on the training set
    for train_idx_inner, val_idx_inner in inner_cv.split(X_train):
        X_train_inner, X_val_inner = X_train[train_idx_inner], X_train[val_idx_inner]
        y_train_inner, y_val_inner = y_train[train_idx_inner], y_train[val_idx_inner]
        model.fit(X_train_inner, y_train_inner)
        y_pred_inner = model.predict(X_val_inner)
        inner_rmse = np.sqrt(mean_squared_error(y_val_inner, y_pred_inner))
        inner_rmse_scores.append(inner_rmse)

    # Calculate the mean RMSE across inner folds
    outer_rmse = np.mean(inner_rmse_scores)
    rmse_scores.append(outer_rmse)


# Calculate the mean RMSE across outer folds
mean_rmse = np.mean(rmse_scores)
print(f"Mean RMSE (nested CV): {mean_rmse:.4f}")

cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
mean_cv_rmse = np.sqrt(-np.mean(cv_scores))
print(f"Mean RMSE (regular CV): {mean_cv_rmse:.4f}")

#compare with resubstution
model.fit(X, y)
y_pred = model.predict(X)
resubstitution_rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Mean RMSE (resubstution): {resubstitution_rmse:.4f}")




Mean RMSE (nested CV): 0.3088
Mean RMSE (regular CV): 15.1747
Mean RMSE (resubstution): 0.2578
